In [1]:
import os,sys

In [2]:
os.chdir('../')

In [3]:
pwd

'e:\\gemstone'

In [4]:
import pandas as pd
import numpy as np
from src.utils.utils import read_yaml,create_dir,save_obj,model_evaluate
from src.constant.ymal_path import *
from dataclasses import dataclass

In [21]:
from pathlib import Path

@dataclass
class ModelTrainConfig:
    dir:Path
    train_arr:Path
    test_arr:Path
    model: Path

In [27]:
class ConfigManager:
    def __init__(self,
                    config_filr_path=Config_ymal_file_path,
                    prams_file_path=Param_ymal_file_path
                    ):
            self.config=read_yaml(config_filr_path)
            self.params=read_yaml(prams_file_path)
           

            create_dir([self.config.artifacts_root])

    def get_model_train_config(self):
          config=self.config.model_trainer
          
         

          create_dir([config.dir])

          model_train_config=ModelTrainConfig(
                dir=config.dir,
                train_arr=config.train_arr,
                test_arr=config.test_arr,
                model=config.model
                
          )
          return model_train_config

In [28]:
from src.logging.logger import logging
from src.exception.exception import CustomException

In [29]:
from sklearn.linear_model import LinearRegression,Lasso,ElasticNet,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [32]:
class ModelTrain:
    def __init__(self,config:ModelTrainConfig) -> None:
        self.config=config

    def initate_model_trainer(self):
        try:
            train_array=np.load(self.config.train_arr)
            test_array=np.load(self.config.test_arr)

            x_train, y_train, x_test, y_test = (
                train_array[:,:-1],
                train_array[:,-1],
                test_array[:,:-1],
                test_array[:,-1])
            models={
                'Liner Regression':LinearRegression(),
                'Lasso Regression': Lasso(),
                'ElasticNet Regression': ElasticNet(),
                'Ridge Regeassor' : Ridge(),
                'RandomForest Regressor': RandomForestRegressor(),
                'DecisionTree Regressor' : DecisionTreeRegressor()
                }
            model_report:dict=model_evaluate(x_train=x_train,y_train=y_train,x_test=x_test,y_test=y_test,models=models)   
            print( model_report)

            print( model_report)

            
            logging.info(f'Model Report : {model_report}')

            print('\n====================================================================================\n')

            # 6. Find the best model
            best_model_score=max(sorted(model_report.values()))

            
            best_model_name = list(model_report.keys())[
                list(model_report.values()).index(best_model_score)
            ]

            best_model = models[best_model_name]

            print(f'Best Model Found , Model Name : {best_model_name} , Score : {best_model_score}')
            print('\n====================================================================================\n')
            logging.info(f'Best Model Found , Model Name : {best_model_name} , Score : {best_model_score}')


            save_obj(
                file_path=self.config.model,
                obj=best_model
            )
            return self.config.model
            
        except CustomException as e:
            logging.info(f'Error cooured {str(e)}')
            raise CustomException(sys,e)    


In [33]:
try:
    config=ConfigManager()
    model_train_config=config.get_model_train_config()
    model_train=ModelTrain(config=model_train_config)
    model_train.initate_model_trainer()
except CustomException as e:
            logging.info(f'Error cooured {str(e)}')
            raise CustomException(sys,e)       

{'Liner Regression': ['Accuracy: 93.73%  MAE: 674.74%  MSE: 1023412.13%'], 'Lasso Regression': ['Accuracy: 93.73%  MAE: 675.89%  MSE: 1023864.61%'], 'ElasticNet Regression': ['Accuracy: 85.53%  MAE: 1067.72%  MSE: 2361647.57%'], 'Ridge Regeassor': ['Accuracy: 93.73%  MAE: 674.77%  MSE: 1023382.10%'], 'RandomForest Regressor': ['Accuracy: 97.79%  MAE: 308.63%  MSE: 360519.05%'], 'DecisionTree Regressor': ['Accuracy: 95.79%  MAE: 423.95%  MSE: 686727.63%']}
{'Liner Regression': ['Accuracy: 93.73%  MAE: 674.74%  MSE: 1023412.13%'], 'Lasso Regression': ['Accuracy: 93.73%  MAE: 675.89%  MSE: 1023864.61%'], 'ElasticNet Regression': ['Accuracy: 85.53%  MAE: 1067.72%  MSE: 2361647.57%'], 'Ridge Regeassor': ['Accuracy: 93.73%  MAE: 674.77%  MSE: 1023382.10%'], 'RandomForest Regressor': ['Accuracy: 97.79%  MAE: 308.63%  MSE: 360519.05%'], 'DecisionTree Regressor': ['Accuracy: 95.79%  MAE: 423.95%  MSE: 686727.63%']}


Best Model Found , Model Name : RandomForest Regressor , Score : ['Accuracy: 9